In [1]:
from urllib.request import urlopen
import requests
import json
import pandas as pd
from pandas import DataFrame
import datetime

## get data

### driver

In [2]:
driver = pd.DataFrame(requests.get('https://api.openf1.org/v1/drivers').json())

/lib/python3.11/site-packages/urllib3/connectionpool.py:1101: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.openf1.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [3]:
cols_driver = ["driver_number", "meeting_key", "session_key", "full_name", "team_name"] 

In [4]:
driver = driver[cols_driver]

### session

In [5]:
session = pd.DataFrame(requests.get('https://api.openf1.org/v1/sessions').json())

/lib/python3.11/site-packages/urllib3/connectionpool.py:1101: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.openf1.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [8]:
cols_session = ["meeting_key", "session_key", "session_type", "date_start", "date_end", "year"]

In [9]:
session = session[cols_session]

## data check

### driver

In [10]:
driver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   driver_number  3143 non-null   int64 
 1   meeting_key    3143 non-null   int64 
 2   session_key    3143 non-null   int64 
 3   full_name      3143 non-null   object
 4   team_name      2186 non-null   object
dtypes: int64(3), object(2)
memory usage: 98.3+ KB


In [11]:
driver[driver["team_name"].isnull() == True]

,driver_number,meeting_key,session_key,full_name,team_name
40,20,1140,9222,Kevin MAGNUSSEN,None
133,1,1141,7953,Max VERSTAPPEN,None
134,2,1141,7953,Logan SARGEANT,None
135,4,1141,7953,Lando NORRIS,None
136,10,1141,7953,Pierre GASLY,None
...,...,...,...,...,...
2202,55,1226,9192,Carlos SAINZ,None
2203,63,1226,9192,George RUSSELL,None
2204,77,1226,9192,Valtteri BOTTAS,None
2205,81,1226,9192,Oscar PIASTRI,None


In [12]:
############################################## 결측치 처리

driver['team_name'] = driver.groupby('full_name')['team_name'].transform(lambda x: x.ffill().bfill())

### session

In [15]:
session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   meeting_key   157 non-null    int64 
 1   session_key   157 non-null    int64 
 2   session_type  157 non-null    object
 3   date_start    157 non-null    object
 4   date_end      157 non-null    object
 5   year          157 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 5.6+ KB


In [17]:
session["year"].unique()

array([2023, 2024], dtype=int64)

### to CSV

In [14]:
# driver
driver.to_csv('./driver.csv', sep=',', na_rep='Null', index=False)

In [18]:
# session
session.to_csv('./session.csv', sep=',', na_rep='Null', index=False)

In [19]:
pd.merge(session, driver, how='left', on=['session_key', 'meeting_key'])

,meeting_key,session_key,session_type,date_start,date_end,year,driver_number,full_name,team_name
0,1140,9222,Practice,2023-02-23T07:00:00+00:00,2023-02-23T16:30:00+00:00,2023,1,Max VERSTAPPEN,Red Bull Racing
1,1140,9222,Practice,2023-02-23T07:00:00+00:00,2023-02-23T16:30:00+00:00,2023,2,Logan SARGEANT,Williams
2,1140,9222,Practice,2023-02-23T07:00:00+00:00,2023-02-23T16:30:00+00:00,2023,4,Lando NORRIS,McLaren
3,1140,9222,Practice,2023-02-23T07:00:00+00:00,2023-02-23T16:30:00+00:00,2023,10,Pierre GASLY,Alpine
4,1140,9222,Practice,2023-02-23T07:00:00+00:00,2023-02-23T16:30:00+00:00,2023,14,Fernando ALONSO,Aston Martin
...,...,...,...,...,...,...,...,...,...
3138,1237,9531,Race,2024-06-09T18:00:00+00:00,2024-06-09T20:00:00+00:00,2024,44,Lewis HAMILTON,Mercedes
3139,1237,9531,Race,2024-06-09T18:00:00+00:00,2024-06-09T20:00:00+00:00,2024,55,Carlos SAINZ,Ferrari
3140,1237,9531,Race,2024-06-09T18:00:00+00:00,2024-06-09T20:00:00+00:00,2024,63,George RUSSELL,Mercedes
3141,1237,9531,Race,2024-06-09T18:00:00+00:00,2024-06-09T20:00:00+00:00,2024,77,Valtteri BOTTAS,Kick Sauber
